In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
# Item Rando:
# 26 'randomizeable' items
# 100 'randomizeable' drop locations
# idea 1: location : item_id
# ~ 500 bits (locations * log(items))
# idea 2: item : location_id
# ~ 700 bits (locations * log(locations))
# idea 3: location : common_item_id
# major_item : location_id
# 300 bits + 126 bits

In [3]:
# Door Rando:
# ~550 doors
# idea 1: door: door_id
# 550 * log(550) = 5500 bits
# idea 2: left_door : right_door_id
# top_door : bottom_door_id
# 225 * log(225) = 2040
# idea 3:
# X, Y
# overworld is ~ 128 x 128
# 14 bits per room * 255 rooms = 3570

In [4]:
import sys
sys.path.append("../..")

In [5]:
from encoding.parse_rooms import parse_rooms, parse_exits, dictify_rooms

In [6]:
from functools import cache
from itertools import combinations

In [7]:
rooms = parse_rooms("../../encoding/dsl/rooms_for_alloy.txt")
exits = parse_exits("../../encoding/dsl/exits_for_alloy.txt")
design = dictify_rooms(rooms, exits)

In [8]:
from rom_tools.rom_manager import RomManager
from abstraction_validation.sm_paths import *

In [9]:
from bdds.bdd_core import *
from bdds.node_bdds import *

In [10]:
from functools import reduce

In [11]:
all_nodes = []
for r, room in rooms.items():
    for node in room.graph.name_node.keys():
        all_nodes.append(node)
node_ids = {n:i for i,n in enumerate(all_nodes)}

In [12]:
# Issue with design translation via chopping off the last _
node_ids["Spore_Spawn_Spawn"] = node_ids["Spore_Spawn_Spore_Spawn"]
node_ids["Golden_Torizo_Torizo"] = node_ids["Golden_Torizo_Golden_Torizo"]
node_ids["Mother_Brain_Brain"] = node_ids["Mother_Brain_Mother_Brain"]

In [13]:
def loc_id(room_name, node_name, when="prev"):
    node_id = node_ids[f"{room_name}_{node_name}"]
    return f"node_id_{when} = {node_id}"

In [14]:
major_items = ["MB", "B", "SPB", "G", "SA", "V", "GS", "SB", "HJ", "CB", "WB", "PLB", "Spazer", "XR", "IB", "SJ"]
minor_items = ["M", "S", "PB", "E", "RT"]
rando_items = ["B", "MB", "PB", "SPB", "S", "M", "G", "SA", "V", "GS", "SB", "HJ", "CB", "WB", "E", "PLB", "Spazer", "RT", "XR", "IB", "SJ"]
#TODO: which items are unique?
#TODO: one-hot encoding

In [15]:
set(rando_items) == set(major_items) | set(minor_items)

True

In [16]:
len(rando_items)

21

In [17]:
context = mk_context_id(node_ids)

In [18]:
def mk_varstr(node, item):
    return f"drop_{node}_{item}"

In [19]:
#TODO: make this more efficient
@cache
def item_transitions(item_gained=None):
    if item_gained is None:
        return "(items_unchanged)"
    clauses = []
    for i in design["Items"] | design["Bosses"]:
        if i == item_gained:
            clause = f"{i}_prev < {i}_next"
        else:
            clause = f"{i}_prev = {i}_next"
        clauses.append(clause)
    return " & ".join(clauses)

def itemset_to_str(itemset):
    if len(itemset) == 0:
        return "TRUE"
    else:
        return " & ".join([f"{item}_prev = 1" for item in itemset])

def required_itemsets(itemsets):
    return "(" + " | ".join([itemset_to_str(itemset) for itemset in itemsets]) + ")"

def rando_transitions(room_name, node_name, family, possible_items):
    t = context.false
    node = room_name + "_" + node_name
    for item in possible_items:
        drop_name = mk_varstr(node, item)
        t |= context.add_expr(f"{drop_name} = 1") & context.add_expr(item_transitions(item))
    return t

In [20]:
major_nodes = []
minor_nodes = []
for room_name, room in design["Rooms"].items():
    for node_name, d in room["Drops"].items():
        if d in major_items:
            major_nodes.append(room_name + "_" + node_name)
        if d in minor_items:
            minor_nodes.append(room_name + "_" + node_name)

In [21]:
assert len(major_nodes) == len(major_items)
assert len(minor_nodes) == 100 - len(major_nodes)

In [22]:
drop_nodes = []
for room_name, room in design["Rooms"].items():
    for node_name, d in room["Drops"].items():
        if d in rando_items:
            drop_nodes.append(room_name + "_" + node_name)

In [23]:
assert len(drop_nodes) == 100

In [24]:
design_vars = {}
for node in major_nodes:
    for item in major_items:
        design_vars[mk_varstr(node, item)] = (0,1)
for node in minor_nodes:
    for item in minor_items:
        design_vars[mk_varstr(node, item)] = (0,1)

In [25]:
if len(design_vars) > 0:
    context.declare(**design_vars)

In [26]:
len(context.bdd.vars)

860

In [27]:
n_major_rando = 16
n_minor_rando = 0

major_rando_nodes = major_nodes[:n_major_rando]
minor_rando_nodes = minor_nodes[:n_minor_rando]

original_design = context.true
for room_name, room in design["Rooms"].items():
    for node_name, d in room["Drops"].items():
        name = room_name + "_" + node_name
        if d in major_items:
            if name in major_nodes and not(name in major_rando_nodes):
                original_design &= context.add_expr(mk_varstr(name, d) + " = 1")
        if d in minor_items:
            if name in minor_nodes and not(name in minor_rando_nodes):
                original_design &= context.add_expr(mk_varstr(name, d) + " = 1")

In [28]:
# Ensure uniqueness

# Every item location drops one item
drop_uniqueness = context.true
for node in tqdm(major_nodes):
    drop_uniqueness &= context.add_expr(" + ".join([mk_varstr(node, item) for item in major_items]) + " = 1")
for node in tqdm(minor_nodes):
    drop_uniqueness &= context.add_expr(" + ".join([mk_varstr(node, item) for item in minor_items]) + " = 1")

# Every major item is dropped exactly once
major_item_uniqueness = context.true
for item in tqdm(major_items):
    major_item_uniqueness &= context.add_expr(" + ".join([mk_varstr(node, item) for node in major_nodes]) + " = 1")

#uniqueness = context.true
#for node, node2 in combinations(major_drop_nodes, 2):
#    uniqueness &= context.add_expr(f"major_drop_{node} != major_drop_{node2}")
# Sum is faster to compute than O(n^2) !=s
uniqueness = drop_uniqueness & major_item_uniqueness

100%|███████████████████████████████████████████████████████████████████████████████████| 16/16 [00:13<00:00,  1.17it/s]


In [29]:
uniqueness.dag_size

1901303

In [30]:
def combine_bdd_group(bdds):
    return reduce(lambda x,y: x|y, bdds, context.false)

In [31]:
# Build individual BDDs
room_bdds = []
door_bdds = []
for room_name, room in tqdm(design['Rooms'].items()):
    links = []
    for node_name in room['Nodes']:
        s = loc_id(room_name, node_name) + " & " + loc_id(room_name, node_name, when="next") + " & " + item_transitions()
        links.append(context.add_expr(s, with_ops=True))
        if node_name in room['Drops']:
            if room_name + "_" + node_name in major_rando_nodes:
                s = context.add_expr(loc_id(room_name, node_name) + " & " + loc_id(room_name, node_name, when="next")) & rando_transitions(room_name, node_name, "major", major_items)
            elif room_name + "_" + node_name in minor_rando_nodes:
                s = context.add_expr(loc_id(room_name, node_name) + " & " + loc_id(room_name, node_name, when="next")) & rando_transitions(room_name, node_name, "minor", minor_items)
            else:
                s = context.add_expr(loc_id(room_name, node_name) + " & " + loc_id(room_name, node_name, when="next") + " & " + item_transitions(room['Drops'][node_name]), with_ops=True)
            links.append(s)
    for node_name, door in room['Doors'].items():
        d = loc_id(room_name, node_name) + " & " + loc_id(door['Room'], door['Node'], when="next") + " & " + item_transitions()
        door_bdds.append(context.add_expr(d, with_ops=True))
    for node_name, edges in room['Edges'].items():
        for edge in edges:
            other_node_name = edge['Terminal']
            s = loc_id(room_name, node_name) + " & " + loc_id(room_name, other_node_name, when="next") + " & " + required_itemsets(edge['Requirements']) + " & " + item_transitions()
            links.append(context.add_expr(s, with_ops=True))
    room_bdd = combine_bdd_group(links)
    #print(room_bdd.count())
    room_bdds.append(room_bdd)

100%|█████████████████████████████████████████████████████████████████████████████████| 255/255 [00:14<00:00, 17.91it/s]


In [32]:
doors_bdd = combine_bdd_group(door_bdds)
rooms_bdd = combine_bdd_group(room_bdds)
all_bdd = doors_bdd | rooms_bdd

In [33]:
all_bdd.dag_size

28688

In [34]:
all_bdd.count()
context.exist(prevs, all_bdd).count()

349.0

In [35]:
len(all_bdd.support)

348

In [36]:
items_unchanged = context.add_expr("items_unchanged", with_ops=True)

In [37]:
item_nexts = [k for k in item_vars if k.endswith("_next")]

In [38]:
def get_reachable_nav_steps(trans, context, start_bdd):
    n = 0
    #trans_local = context.exist(item_nexts + ["Ceres_Ridley_next"], all_bdd & ~context.add_expr("node_id_prev = node_id_next"))
    trans_local = context.exist(item_nexts, trans & items_unchanged)
    #trans_local = trans & items_unchanged
    covered = start_bdd
    covered_last = context.false
    while covered != covered_last:
        covered_last = covered
        # Inner while loop for no-pickup navigation
        covered_local = covered
        covered_local_last = context.false
        while covered_local != covered_local_last:
            covered_local_last = covered_local
            fringe_local = context.exist(["node_id_prev"], covered_local & trans_local)
            covered_local |= context.let({"node_id_next": "node_id_prev"}, fringe_local)
            #print(context.support(covered_local))
            #print("\t", covered_local.dag_size)
        covered |= covered_local
        # Unrestricted step
        fringe = context.exist(prevs, covered & trans)
        covered |= context.let(next_to_prev, fringe)
        #print(n, covered.dag_size)
        print(covered.dag_size)
        n+=1
    return covered

In [39]:
def get_reachable_id(trans_norule, context, start_bdd):
    n = 0
    covered_p = start_bdd
    covered_last_p = context.false
    while covered_p != covered_last_p:
        covered_last_p = covered_p
        fringe_n = context.exist(prevs, covered_p & trans_norule)
        covered_p |= context.let(next_to_prev, fringe_n)
        print(n, covered_p.dag_size)
        n+=1
    return covered_p

In [40]:
def mk_closure(trans, context, max_steps=float("inf")):
    n = 0
    closure = trans
    closure_last = context.false
    while closure != closure_last and n < max_steps:
        closure_last = closure
        closure_prev_temp = context.let(next_to_temp, closure_last)
        closure_temp_next = context.let(prev_to_temp, closure_last)
        closure |= context.exist(temps, closure_prev_temp & closure_temp_next)
        print(n, closure.dag_size)
        n+=1
    closure_square = closure
    return closure_square

In [41]:
def get_reachable_nav_closure(trans, context, start_bdd):
    n = 0
    #trans_local = context.exist(item_nexts + ["Ceres_Ridley_next"], all_bdd & ~context.add_expr("node_id_prev = node_id_next"))
    trans_local = trans & items_unchanged
    trans_local = mk_closure(trans_local, context)
    trans = trans_local | trans
    #trans_local = trans & items_unchanged
    covered = start_bdd
    covered_last = context.false
    while covered != covered_last:
        covered_last = covered
        fringe = context.exist(prevs, covered & trans)
        covered |= context.let(next_to_prev, fringe)
        #print(n, covered.dag_size)
        n+=1
    return covered

In [42]:
from data_types import item_set

In [43]:
def mk_itemset_expr(itemset, when="prev"):
    clauses = []
    for i in item_mapping:
        if i in itemset:
            clause = f"{i}_{when} = 1"
        else:
            clause = f"{i}_{when} = 0"
        clauses.append(clause)
    return " & ".join(clauses)

In [44]:
start_bdd = context.add_expr("node_id_prev = 0 & " + mk_itemset_expr(item_set.ItemSet()))

In [45]:
all_items_bad = context.true
for drop in design_vars.keys():
    all_items_bad &= context.add_expr(f"{drop} < 4")

In [46]:
context.bdd.configure()

{'reordering': False,
 'garbage_collection': True,
 'max_memory': 18446744073709551615,
 'loose_up_to': 6710886,
 'max_cache_soft': 31031296,
 'max_cache_hard': 4294967295,
 'min_hit': 30,
 'max_growth': 1.2,
 'max_swaps': 2000000,
 'max_vars': 1000}

In [47]:
%%time
reachable = get_reachable_nav_steps(all_bdd, context, start_bdd & uniqueness)
#reachable = get_reachable_nav_steps(all_bdd_fast, context, start_bdd & uniqueness)
#reachable = get_reachable_nav_closure(all_bdd, context, start_bdd)

1939047
1939113
1939556
1940155
1940337
2371915
3230218


KeyboardInterrupt: 

In [48]:
reachable.dag_size - 1901303

NameError: name 'reachable' is not defined

In [ ]:
reachable.count()

In [ ]:
context.support(reachable)

In [ ]:
# add to trans 1.844674407372149e+19
# add to start 11939936.0
# 11939936.0

In [ ]:
drop_nodes

In [ ]:
%%time
all_bdd_fast = mk_closure(all_bdd, context, 3)

0 115455


In [ ]:
#%%time
#reachable = get_reachable_id(all_bdd_fast, context, start_bdd)

In [ ]:
#TODO: Iterative Squaring

In [ ]:
end_reachable = context.add_expr("node_id_prev = 1")

In [ ]:
g_wo_s = context.add_expr("G_prev = 1 & S_prev = 0")

In [ ]:
(end_reachable & reachable).count()

In [ ]:
#TODO: ensure no invalid node ids

In [ ]:
get_g_wo_s = context.exist(prevs, reachable & g_wo_s)
get_end = context.exist(prevs, reachable & end_reachable)

In [ ]:
(get_g_wo_s & get_end).dag_size

In [ ]:
for sol in context.pick_iter(get_g_wo_s & get_end):
    td = {}
    for k,v in sol.items():
        if v < len(rando_items):
            td[k] = rando_items[v]
        else:
            td[k] = v
    print(td)

In [ ]:
# Gaussian distribution over shortest path times
# Action n-grams
# Major/Minor rando
# nx.draw layout

In [ ]:
#TODO: Region-based or item-collection based cuts
# Saturate the space ahead-of-time